# Extraction of specific entities in the raw CV

The aim of this notebook is to build a Dataframe that will extract some specific information (e-mail, numeric numbers that are date of birth, period of time, some components of address or zipcode). A processed text will then be generated that will not contain anymore this numeric values that can add errors in the next entities recognition

In [61]:
#Reuse of the function from the previous notebook to retrieve the CV text and build the Dataframe
def para2text(p):
    rs = p._element.xpath('.//w:t')
    return u" ".join([r.text for r in rs])

In [62]:
import os
import docx
title = os.listdir('Resumefromblob')
text = [para2text(docx.Document('Resumefromblob/' + filename)) for filename in os.listdir('Resumefromblob')]
idb=[]
for i in range(len(text)):
    idb.append(i+1)
print(idb)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


In [63]:
import json
title_text=[{"id":a,"title": b, "text": c} for a,b,c in zip(idb, title, text)]
js= json.dumps(title_text)
doc={}
doc['documents']= title_text
#json string containing id, title of the CV and the content (text) of the CV
json_doc = json.dumps(doc) 

text_id=[{"id":a,"text": b} for a,b in zip(idb, text)]
docbis={}
docbis['documents']= text_id
#json string containing id, title of the CV and the content (text) of the CV
json_docbis = json.dumps(docbis) 
#convert in dictionary
docs=json.loads(json_docbis)

documents=json.loads(json_doc)

In [126]:
#Building of the 1st columns of the dataframe
import pandas as pd
idb,title,text=[],[],[]
for doc in documents['documents']:
    idb.append(doc['id'])
    title.append(doc['title'])
    text.append(doc['text'])
df=pd.DataFrame([title,text], columns=idb, index=['Title','Text']).T
df.head()

,Title,Text
1,1805-TQL-CV-PastryChef.docx,LY Thanh Quan PASTRY CHEF Personal information...
2,CV-MathildeGABRIEL.docx,Mathilde GABRIEL PROFILE Age : 26 Birthplace :...
3,CV-RaphaëlSahakian-VE.docx,OBJECTIVE Management School student currently ...
4,CV_Louise.docx,334A Pasir Panjang Road #05-18 117382 Singap...
5,CVanglo.docx,Louis-Paul Courouble 1 rue des Flandres 59155 ...


In [65]:
#Test if all the text is in the Text column dataframe
df['Text'][13]

'L UCIE  PROST lucie.prost06@gmail.com +6586783019 Singapore LIFE SCIENCES ENGINEER Open to new opportunities in scientific and marketing project management E DUCATION 2012-2015  Engineering degree   from   ENSCM (Montpellier, France)   i.e. National Graduate School of Chemistry, a French   highly selective academic Institute of Science and Engineering.  Graduate Major:   Cosmetology and Pharmacology 2009-2011  Two years in Preparatory Classes at ENSCCF (Clermont-Ferrand, France) W ORK  E XPERIENCE 06/2017-03/2018 (09 months)  Sabbatical year traveling throughout South East Asia to discover and define in which   country we would settle down → Indonesia, Singapore, Malaysia, Thailand, Cambodia, Laos, Vietnam, Philippines, Sri Lanka 09/2015-05/2017 (18 months)  Business Manager , Pharmaceutical sector, CRO Aixial, Alten, Belgium Key Account Manager for GlaxoSmithKline (GSK) and for all clinical activities of Aixial, an Alten company Business: cold calling, client meetings, understanding 

## Why do we need to remove some values of the raw text ?

### The segmentation example !

In [88]:
#!pip install wordsegment

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [89]:
from wordsegment import load, segment, clean
load()
segment('thi sisatest')

['this', 'is', 'a', 'test']

In [90]:
segment(df['Text'][13])
#problem with separation of email adress and number of tel
#problem with number 20122015

['lucie',
 'prost',
 'lucie',
 'pro',
 'st06gmailcom6586783019',
 'singapore',
 'life',
 'sciences',
 'engineer',
 'open',
 'to',
 'new',
 'opportunities',
 'in',
 'scientific',
 'and',
 'marketing',
 'project',
 'management',
 'education',
 '20122015',
 'engineering',
 'degree',
 'from',
 'en',
 'scm',
 'montpellier',
 'france',
 'ie',
 'national',
 'graduate',
 'school',
 'of',
 'chemistry',
 'a',
 'french',
 'highly',
 'selective',
 'academic',
 'institute',
 'of',
 'science',
 'and',
 'engineering',
 'graduate',
 'major',
 'cosmetology',
 'and',
 'pharmacology',
 '20092011',
 'two',
 'years',
 'in',
 'preparatory',
 'classes',
 'a',
 'tens',
 'ccf',
 'clermont',
 'ferrand',
 'france',
 'work',
 'experience',
 '06201703201809',
 'months',
 'sabbatical',
 'year',
 'traveling',
 'throughout',
 'south',
 'east',
 'asia',
 'to',
 'discover',
 'and',
 'define',
 'in',
 'which',
 'country',
 'we',
 'would',
 'settle',
 'down',
 'indonesia',
 'singapore',
 'malaysia',
 'thailand',
 'cambod

There, characters of the email address and some numbers have generated problems and strange segmentation in the text. We should capture email adress, telephone number and some specific period and date that can answer some specific criteria so that this does not compromise the segmmentation

The first idea would be to extract some numbers from the resume. But these numbers  could mean anything, and then we could lose some information they would provide if we just erase it from the text

In [ ]:
import re

In [67]:
#extract number from resume
number=[]
for i in range(len(text)):
    a=list(map(int, re.findall(r'\d+', df['Text'][i+1])))
    number.append(a)
print(number)

[[483, 13, 14, 248368, 65, 8393, 2616, 1001, 2016, 1994, 2016, 2017, 150, 2016, 2016, 2015, 2016, 1728, 80, 5, 20, 7, 6, 8, 5, 9, 4, 1, 2, 1, 2, 5, 5, 6, 7, 8, 8, 7, 6, 1, 2, 5, 8, 6, 7, 1], [26, 92, 65, 9139, 1049, 16, 26, 1, 88661, 2017, 2017, 3, 6270, 2018, 3, 2016, 2017, 6, 54571718, 2013, 2016, 6300, 2015, 18, 2015, 2016, 6, 6300, 2010, 2015, 10, 6300, 2014, 2016, 2013, 2014, 2011, 2013, 2010, 17, 2009, 92], [5, 6, 2018, 1, 657, 677, 1, 1, 2017, 2018, 2017, 123, 2016, 4, 2, 2013, 2019, 2013, 2015, 2010, 2013, 33601392325, 6, 2, 2107], [334, 5, 18, 117382, 65, 84387193, 334, 5, 18, 117382, 65, 84387193, 2016, 2019, 2015, 2018, 2016, 2013, 2015, 2010, 2013, 2014, 21, 2017, 6, 2017, 1, 2016, 2017, 5, 1, 2016, 1, 1, 1, 8, 500, 2016, 6, 2015, 1, 2015, 1, 2015, 1, 980, 990, 5, 0, 17], [1, 59155, 6, 31, 21, 19, 67, 2017, 2018, 2016, 2017, 2014, 2016, 59, 2014, 2018, 2017, 2017, 59, 2016, 2017, 2016, 2016, 2015, 2015, 59], [5, 31400, 33, 6, 45, 86, 62, 34, 31, 21, 3, 2017, 2018, 2014, 201

## Extraction of different entities and new columns in the dataframe

### E-mail address and creation of 'New-Text' column

In [127]:
#extract email address from resume
email=[]
new_text=[]
for i in range(len(text)):
    new_text.append(df['Text'][i+1])
    a=re.findall(r'[\w\.-]+@[\w\.-]+', df['Text'][i+1])
    email.append(a)
    for j in a:
        new_text[i]=new_text[i].replace(j,'')
#remove duplicate
for i in range(len(text)):
    if len(email[i])>1:
           email[i]=set(email[i])
#print((df['Text'][1]).strip( 'a' ))
print(new_text[1])

Mathilde GABRIEL PROFILE Age : 26 Birthplace : Saint Cloud (92) Nationality : French Single Permis B YPY Member CONTACTS Mob. +65 9139 1049  16 Spottiswoode Park Road, Spottiswoode Suites, #26.01 088661 Singapore LANGUAGES French   +++++ English ++++ Spanish +++ SKILLS Creativity Dynamic Organized Team Hard- Quick Work Working Learner COMPETENCES Photoshop / Lightroom Wordpress iWorks : Pages / Keynote / Numbers Microsoft Office HOBBIES ABOUT ME Newly arrived to Singapore in October 2017 , I am a motivated, creative, and pas-sionate Communication & Marketing professional. I would like to offer my skills and energy to a large renowned and dynamic company.  WORK EXPERIENCE April 2017 - Now :  MARKETING MANAGER Lumion 3D  - Marina Baie des Anges - 06270 Villeneuve-Loubet Marketing Strategy  : Define all the major communication strategic lines - Marketing Plan & budgets for 2018 - Develop and Implement the planned marketing, communication and sales measures. Website  : Redesign the Wordpre

In [128]:
df['Email']=email
df['New_Text']=new_text
df

,Title,Text,Email,New_Text
1,1805-TQL-CV-PastryChef.docx,LY Thanh Quan PASTRY CHEF Personal information...,[thanhquanly1001@gmail.com],LY Thanh Quan PASTRY CHEF Personal information...
2,CV-MathildeGABRIEL.docx,Mathilde GABRIEL PROFILE Age : 26 Birthplace :...,[gabriel.mathilde@gmail.com],Mathilde GABRIEL PROFILE Age : 26 Birthplace :...
3,CV-RaphaëlSahakian-VE.docx,OBJECTIVE Management School student currently ...,[raphael.sahakian@gmail.com],OBJECTIVE Management School student currently ...
4,CV_Louise.docx,334A Pasir Panjang Road #05-18 117382 Singap...,{louise.picard@student.ecp.fr},334A Pasir Panjang Road #05-18 117382 Singap...
5,CVanglo.docx,Louis-Paul Courouble 1 rue des Flandres 59155 ...,[courouble.louispaul@orange.fr],Louis-Paul Courouble 1 rue des Flandres 59155 ...
6,CVenanglais.docx,Hugo TORRENT 5 AVENUE DE LA MARCAISSONNE ...,[hugo.trrt31@gmail.com],Hugo TORRENT 5 AVENUE DE LA MARCAISSONNE ...
7,CVF&B.docx,Giulia BALEGNO Current Address Les Roches Camp...,"{giulia.balegno@lesroches.ch, balegnogiulia@gm...",Giulia BALEGNO Current Address Les Roches Camp...
8,CVjanvierpierrick.docx,-81 rue David D’Angers 49130 les Ponts de Cé. ...,[Pierrick.janvier@essca.eu],-81 rue David D’Angers 49130 les Ponts de Cé. ...
9,oussama-CVENG.docx,KHOMSI Oussama SOFTWARE CONCEPTION & DEVELOPME...,[okhomsielhaddad@gmail.com],KHOMSI Oussama SOFTWARE CONCEPTION & DEVELOPME...
10,PaulMATHONCVinternship.docx,Paul MATHON BUSINESS STUDENT LOOKINF FOR AN IN...,[paulmathon0@gmail.com],Paul MATHON BUSINESS STUDENT LOOKINF FOR AN IN...


### Phone number : some trials

In [129]:
#extract phone number
    #here, the string may start with + or ( symbol
    #it has to be followed by a number between 0-9 or a space
    #it has to end with a number between between 0-9
    #it may contain 0-9 (space) .-() in the middle
phone_or_date=[]
for i in range(len(text)):
    a=re.findall(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]', df['Text'][i+1])
    phone_or_date.append(a)
#remove duplicate
for i in range(len(text)):
    if len(phone_or_date[i])>1:
           phone_or_date[i]=set(phone_or_date[i])
print(phone_or_date)
#in tbe expression the r'' is used to use raw string notation
#find a rule to distinguish phone number from date
#find a rule that extracts the #xx-xx unit number and postcode

#extract unit or postcode from the phone_or_date
    #here, the string has to start with # symbol
    #it has to be followed by a number between 0-9 or a space
    #it has to end with a number between between 0-9
    #it may contain 0-9 (space) .-() in the middle
unit_postcode_singap=[]
for i in range(len(text)):
    a=re.findall(r'[\#][0-9(\)][0-9 .\-\(\)]{4,}[0-9 \,]', df['Text'][i+1])
    unit_postcode_singap.append(a)
#remove duplicate
for i in range(len(text)):
    if len(unit_postcode_singap[i])>1:
           unit_postcode_singap[i]=set(unit_postcode_singap[i])
print(unit_postcode_singap)

[{'+65 8393 2616', '5 5 6 7 8 8 7 6'}, {'2013 - 2014', '+65 9139 1049', '26.01 088661', '2011 - 2013', '2014 - 2016'}, ['+33601392325'], {'2010 - 2013', '+65 84387193', '5-18 117382', '2013 - 2015'}, ['6.31.21.19.67'], {'+33) 6 45 86 62 34', '2017 - 2018', '2014 - 2016', '2016 - 2017'}, ['+41 78 649 66 98'], ['6 74 24 73 74'], {'15.08.2014', '26.07.1990', '30.07.2009', '1.10.2011 01.06.2013', '15.03.2014', '15.09.2011', '17.03.2014', '1.07.2013 06.09.2013', '+33 6 69 65 81 79', '30.07.2008', '15.09.2008 1', '25.09.2013', '15.09.2007', '11.09.2009'}, ['+33 (0)6 86 18 04 57'], {'+33 6 58 26 37 75', '(2014 - 2016'}, ['+336 07 33 80 54'], ['+6586783019']]
[['#13-14,'], ['#26.01 088661 '], [], {'#05-18 117382 '}, [], [], [], [], [], [], [], [], []]


### Some unit code and postcode

In [130]:
#easier version of extraction of unit or postcode
unit_postcode_singap=[]
new_text=[]
for i in range(len(text)):
    new_text.append(df['New_Text'][i+1])
    a=re.findall(r'#[-,.\d ()]{4,}', df['New_Text'][i+1])
    unit_postcode_singap.append(a)
    for j in a:
        new_text[i]=new_text[i].replace(j,'')
#remove duplicate
for i in range(len(text)):
    if len(unit_postcode_singap[i])>1:
           unit_postcode_singap[i]=set(unit_postcode_singap[i])
print(unit_postcode_singap)

[['#13-14, '], ['#26.01 088661 '], [], {'#05-18 117382 '}, [], [], [], [], [], [], [], [], []]


In [131]:
df['Unit_postcode']=unit_postcode_singap
df['New_Text']=new_text
df

,Title,Text,Email,New_Text,Unit_postcode
1,1805-TQL-CV-PastryChef.docx,LY Thanh Quan PASTRY CHEF Personal information...,[thanhquanly1001@gmail.com],LY Thanh Quan PASTRY CHEF Personal information...,"[#13-14, ]"
2,CV-MathildeGABRIEL.docx,Mathilde GABRIEL PROFILE Age : 26 Birthplace :...,[gabriel.mathilde@gmail.com],Mathilde GABRIEL PROFILE Age : 26 Birthplace :...,[#26.01 088661 ]
3,CV-RaphaëlSahakian-VE.docx,OBJECTIVE Management School student currently ...,[raphael.sahakian@gmail.com],OBJECTIVE Management School student currently ...,[]
4,CV_Louise.docx,334A Pasir Panjang Road #05-18 117382 Singap...,{louise.picard@student.ecp.fr},334A Pasir Panjang Road Singapore Singapore ...,{#05-18 117382 }
5,CVanglo.docx,Louis-Paul Courouble 1 rue des Flandres 59155 ...,[courouble.louispaul@orange.fr],Louis-Paul Courouble 1 rue des Flandres 59155 ...,[]
6,CVenanglais.docx,Hugo TORRENT 5 AVENUE DE LA MARCAISSONNE ...,[hugo.trrt31@gmail.com],Hugo TORRENT 5 AVENUE DE LA MARCAISSONNE ...,[]
7,CVF&B.docx,Giulia BALEGNO Current Address Les Roches Camp...,"{giulia.balegno@lesroches.ch, balegnogiulia@gm...",Giulia BALEGNO Current Address Les Roches Camp...,[]
8,CVjanvierpierrick.docx,-81 rue David D’Angers 49130 les Ponts de Cé. ...,[Pierrick.janvier@essca.eu],-81 rue David D’Angers 49130 les Ponts de Cé. ...,[]
9,oussama-CVENG.docx,KHOMSI Oussama SOFTWARE CONCEPTION & DEVELOPME...,[okhomsielhaddad@gmail.com],KHOMSI Oussama SOFTWARE CONCEPTION & DEVELOPME...,[]
10,PaulMATHONCVinternship.docx,Paul MATHON BUSINESS STUDENT LOOKINF FOR AN IN...,[paulmathon0@gmail.com],Paul MATHON BUSINESS STUDENT LOOKINF FOR AN IN...,[]


### New phone numbers (with country code)

Phone number can be tricky as they do not follow all the same rules. So for the moment, as I have some different numbers in my dataset (French, SIngaporean, other nationalities), I will only capture the phones that have the +(national code) rule.

In [132]:
#all the phone with country code
phone_with_country_code=[]
new_text=[]
for i in range(len(text)):
    new_text.append(df['New_Text'][i+1])
    a=re.findall(r'[\+][0-9 \d ()]{4,21}', df['New_Text'][i+1])
    for k in a:
        if k.startswith("+65") and len(k)>13:
            a.remove(k)
            k=k[:13]
            a.append(k)
    phone_with_country_code.append(a)
    for j in a:
        new_text[i]=new_text[i].replace(j,'')
#remove duplicate
for i in range(len(text)):
    if len(phone_with_country_code[i])>1:
           phone_with_country_code[i]=set(phone_with_country_code[i])
print(phone_with_country_code)

[['+65 8393 2616'], ['+65 9139 1049'], ['+33601392325 '], {'+65 84387193 ', '+65 84387193  '}, [], ['+33) 6 45 86 62 34 '], ['+41 78 649 66 98 '], [], ['+33 6 69 65 81 79 '], ['+33 (0)6 86 18 04 57  '], ['+33 6 58 26 37 75 '], ['+336 07 33 80 54 '], ['+6586783019 ']]


In [133]:
df['Phone_country_code']=phone_with_country_code
#updating new_text
df['New_Text']=new_text
df

,Title,Text,Email,New_Text,Unit_postcode,Phone_country_code
1,1805-TQL-CV-PastryChef.docx,LY Thanh Quan PASTRY CHEF Personal information...,[thanhquanly1001@gmail.com],LY Thanh Quan PASTRY CHEF Personal information...,"[#13-14, ]",[+65 8393 2616]
2,CV-MathildeGABRIEL.docx,Mathilde GABRIEL PROFILE Age : 26 Birthplace :...,[gabriel.mathilde@gmail.com],Mathilde GABRIEL PROFILE Age : 26 Birthplace :...,[#26.01 088661 ],[+65 9139 1049]
3,CV-RaphaëlSahakian-VE.docx,OBJECTIVE Management School student currently ...,[raphael.sahakian@gmail.com],OBJECTIVE Management School student currently ...,[],[+33601392325 ]
4,CV_Louise.docx,334A Pasir Panjang Road #05-18 117382 Singap...,{louise.picard@student.ecp.fr},334A Pasir Panjang Road Singapore Singapore ...,{#05-18 117382 },"{+65 84387193 , +65 84387193 }"
5,CVanglo.docx,Louis-Paul Courouble 1 rue des Flandres 59155 ...,[courouble.louispaul@orange.fr],Louis-Paul Courouble 1 rue des Flandres 59155 ...,[],[]
6,CVenanglais.docx,Hugo TORRENT 5 AVENUE DE LA MARCAISSONNE ...,[hugo.trrt31@gmail.com],Hugo TORRENT 5 AVENUE DE LA MARCAISSONNE ...,[],[+33) 6 45 86 62 34 ]
7,CVF&B.docx,Giulia BALEGNO Current Address Les Roches Camp...,"{giulia.balegno@lesroches.ch, balegnogiulia@gm...",Giulia BALEGNO Current Address Les Roches Camp...,[],[+41 78 649 66 98 ]
8,CVjanvierpierrick.docx,-81 rue David D’Angers 49130 les Ponts de Cé. ...,[Pierrick.janvier@essca.eu],-81 rue David D’Angers 49130 les Ponts de Cé. ...,[],[]
9,oussama-CVENG.docx,KHOMSI Oussama SOFTWARE CONCEPTION & DEVELOPME...,[okhomsielhaddad@gmail.com],KHOMSI Oussama SOFTWARE CONCEPTION & DEVELOPME...,[],[+33 6 69 65 81 79 ]
10,PaulMATHONCVinternship.docx,Paul MATHON BUSINESS STUDENT LOOKINF FOR AN IN...,[paulmathon0@gmail.com],Paul MATHON BUSINESS STUDENT LOOKINF FOR AN IN...,[],[+33 (0)6 86 18 04 57 ]


### Birthdate and some other date

In [136]:
#FIND Birthdate
birthdate=[]
not_date=[]
new_text=[]
for i in range(len(text)):
    new_text.append(df['New_Text'][i+1])
    a=re.findall(r'[\d]{1,2}[/\.][\d]{1,2}[/\.][\d]{4}', df['New_Text'][i+1])
    birthdate.append(a)

#Capture the non birthdate
for i in range(len(text)):
    a=re.findall(r'[\d]{1,2}[/\.][\d]{1,2}[/\.][\d]{4}', df['New_Text'][i+1])
    for k in a:
        if k[-4:].isdigit()==True and int(k[-4:])>2003:
            not_date.append(k)

#Remove the non birthdate from the list of list
for k in not_date:
    for i in range(len(birthdate)):
        if k in birthdate[i]:
            birthdate[i].remove(k)

#remove birth date from new_text
for j in birthdate:
    ind=birthdate.index(j)
    a=str(j).replace("['",'')
    a=a.replace("']",'')
    new_text[ind]=new_text[ind].replace(a,'')


#remove duplicate
for i in range(len(text)):
    if len(birthdate[i])>1:
        birthdate[i]=set(birthdate[i])
print(birthdate)
print(new_text[7])
len(new_text)

#new_text[1]

[[], [], [], [], [], [], [], ['17/02/1997'], ['26.07.1990'], [], [], [], []]
-81 rue David D’Angers 49130 les Ponts de Cé. -Tél : 06 74 24 73 74. -Email :  Né le . Permis B. Je suis dynamique, responsable et à l’écoute. Pierrick JANVIER Curriculum vitae. Formations Septembre 2015 Admission au concours d’entrée en première année à  l’ESSCA  (École supérieure des sciences commerciales d'Angers). Programme des grandes écoles visé par l’état du grade de master, bac +5. Septembre 2014 à juin 2015. Terminale ES  au lycée Jean-Bodin. Obtention du  Baccalauréat. Octobre 2014   :   Validation du   BAFA, stage   d’approfondissement   option : camps itinérants et à   l’étranger Expériences professionnelles Janvier 2017 : actionnaire associé de la Sarl le Blue Monkeys, community manager. En charge de la  communication , et de l ’évènementiel  dans un bar/cabaret culturel. Service en salle, service en bar, gestion des stock, maintenance générale et technique du bar. Mai 2016 : Stage de deux mois au

13

In [137]:
df['Birthdate']=birthdate
df['New_Text']=new_text
df

,Title,Text,Email,New_Text,Unit_postcode,Phone_country_code,Birthdate
1,1805-TQL-CV-PastryChef.docx,LY Thanh Quan PASTRY CHEF Personal information...,[thanhquanly1001@gmail.com],LY Thanh Quan PASTRY CHEF Personal information...,"[#13-14, ]",[+65 8393 2616],[]
2,CV-MathildeGABRIEL.docx,Mathilde GABRIEL PROFILE Age : 26 Birthplace :...,[gabriel.mathilde@gmail.com],Mathilde GABRIEL PROFILE Age : 26 Birthplace :...,[#26.01 088661 ],[+65 9139 1049],[]
3,CV-RaphaëlSahakian-VE.docx,OBJECTIVE Management School student currently ...,[raphael.sahakian@gmail.com],OBJECTIVE Management School student currently ...,[],[+33601392325 ],[]
4,CV_Louise.docx,334A Pasir Panjang Road #05-18 117382 Singap...,{louise.picard@student.ecp.fr},334A Pasir Panjang Road Singapore Singapore ...,{#05-18 117382 },"{+65 84387193 , +65 84387193 }",[]
5,CVanglo.docx,Louis-Paul Courouble 1 rue des Flandres 59155 ...,[courouble.louispaul@orange.fr],Louis-Paul Courouble 1 rue des Flandres 59155 ...,[],[],[]
6,CVenanglais.docx,Hugo TORRENT 5 AVENUE DE LA MARCAISSONNE ...,[hugo.trrt31@gmail.com],Hugo TORRENT 5 AVENUE DE LA MARCAISSONNE ...,[],[+33) 6 45 86 62 34 ],[]
7,CVF&B.docx,Giulia BALEGNO Current Address Les Roches Camp...,"{giulia.balegno@lesroches.ch, balegnogiulia@gm...",Giulia BALEGNO Current Address Les Roches Camp...,[],[+41 78 649 66 98 ],[]
8,CVjanvierpierrick.docx,-81 rue David D’Angers 49130 les Ponts de Cé. ...,[Pierrick.janvier@essca.eu],-81 rue David D’Angers 49130 les Ponts de Cé. ...,[],[],[17/02/1997]
9,oussama-CVENG.docx,KHOMSI Oussama SOFTWARE CONCEPTION & DEVELOPME...,[okhomsielhaddad@gmail.com],KHOMSI Oussama SOFTWARE CONCEPTION & DEVELOPME...,[],[+33 6 69 65 81 79 ],[26.07.1990]
10,PaulMATHONCVinternship.docx,Paul MATHON BUSINESS STUDENT LOOKINF FOR AN IN...,[paulmathon0@gmail.com],Paul MATHON BUSINESS STUDENT LOOKINF FOR AN IN...,[],[+33 (0)6 86 18 04 57 ],[]


### Capturing some period of different format

In [202]:
period1=[]
period2=[]
for i in range(len(text)):
    #new_text.append(df['New_Text'][i+1])
    a=re.findall(r'[\d]{1,2}[/\.][\d]{1,2}[/\.][\d]{4}[0-3 -\(\)][\d]{1,2}[/\.][\d]{1,2}[/\.][\d]{4}', df['New_Text'][i+1])
    period1.append(a)
print(period1)

for i in range(len(text)):
    new_text.append(df['New_Text'][i+1])
    a=re.findall(r'[\d]{4}[0-9 \-\]{1,}[\d]{2,5}', df['New_Text'][i+1])
    period2.append(a)
print(period2)

new_list=[i+j for i, j in zip(period1, period2)]
print(new_list)
print(len(new_list))

print(df['New_Text'][2])

[[], [], [], [], [], [], [], [], ['01.07.2013 06.09.2013', '01.10.2011 01.06.2013'], [], [], [], []]
[['248368 ', '2016 - ', '2016 - ', '2015 - ', '1728  '], ['2017 , ', '2017 - ', '06270 ', '2018 - ', '2016 - ', '54571718 ', '2013 - ', '06300 ', '2015 - ', '06300 ', '2010 - ', '06300 ', '2014 - 20', '2013 - 20', '2011 - 20'], ['2018   '], ['2015  ', '2013 - 20', '2010 - 20', '2014   ', '2017  ', '2016-2017', '2016   ', '2015  '], ['59155 ', '2017-2018', '2016-2017', '2014-2016', '2016-2017'], ['31400 ', '2017 - 20', '2014 - 20', '2014  ', '2017 - 20', '2016 - 20'], [], ['49130 ', '2014   '], ['95170 ', '2013 06', '2013  ', '2011 01', '2013  ', '2014   ', '2008 1', '2008   '], ['1874, ', '2011-2016'], ['34000 ', '2014 - ', '2014 - ', '2011 - ', '2014 - 20'], ['2016-18 ', '2013-15 ', '2012-13 ', '2013-15 ', '2012-13 ', '2010-12 '], ['2012-2015', '2009-2011', '2017-03', '2015-05', '2015-09', '2014-09', '2013-03', '2013-10', '2008, 200']]
[['248368 ', '2016 - ', '2016 - ', '2015 - ', '172

### Extract url (error when there is a /n in the url address)

In [183]:
url=[]
#new_text=[]

for i in range(len(text)):
    #new_text.append(df['New_Text'][i+1])
    a=re.findall(r'https?://[^\s]+', df['New_Text'][i+1])
    url.append(a)

In [182]:
df['Url']=url
df

,Title,Text,Email,New_Text,Unit_postcode,Phone_country_code,Birthdate,Url
1,1805-TQL-CV-PastryChef.docx,LY Thanh Quan PASTRY CHEF Personal information...,[thanhquanly1001@gmail.com],LY Thanh Quan PASTRY CHEF Personal information...,"[#13-14, ]",[+65 8393 2616],[],[]
2,CV-MathildeGABRIEL.docx,Mathilde GABRIEL PROFILE Age : 26 Birthplace :...,[gabriel.mathilde@gmail.com],Mathilde GABRIEL PROFILE Age : 26 Birthplace :...,[#26.01 088661 ],[+65 9139 1049],[],[https://vimeo.com/user54571718]
3,CV-RaphaëlSahakian-VE.docx,OBJECTIVE Management School student currently ...,[raphael.sahakian@gmail.com],OBJECTIVE Management School student currently ...,[],[+33601392325 ],[],[https://www.linkedin.com/in/ra]
4,CV_Louise.docx,334A Pasir Panjang Road #05-18 117382 Singap...,{louise.picard@student.ecp.fr},334A Pasir Panjang Road Singapore Singapore ...,{#05-18 117382 },"{+65 84387193 , +65 84387193 }",[],[]
5,CVanglo.docx,Louis-Paul Courouble 1 rue des Flandres 59155 ...,[courouble.louispaul@orange.fr],Louis-Paul Courouble 1 rue des Flandres 59155 ...,[],[],[],[]
6,CVenanglais.docx,Hugo TORRENT 5 AVENUE DE LA MARCAISSONNE ...,[hugo.trrt31@gmail.com],Hugo TORRENT 5 AVENUE DE LA MARCAISSONNE ...,[],[+33) 6 45 86 62 34 ],[],[]
7,CVF&B.docx,Giulia BALEGNO Current Address Les Roches Camp...,"{giulia.balegno@lesroches.ch, balegnogiulia@gm...",Giulia BALEGNO Current Address Les Roches Camp...,[],[+41 78 649 66 98 ],[],[]
8,CVjanvierpierrick.docx,-81 rue David D’Angers 49130 les Ponts de Cé. ...,[Pierrick.janvier@essca.eu],-81 rue David D’Angers 49130 les Ponts de Cé. ...,[],[],[17/02/1997],[]
9,oussama-CVENG.docx,KHOMSI Oussama SOFTWARE CONCEPTION & DEVELOPME...,[okhomsielhaddad@gmail.com],KHOMSI Oussama SOFTWARE CONCEPTION & DEVELOPME...,[],[+33 6 69 65 81 79 ],[26.07.1990],[https://fr.linkedin.com/in/oussamak]
10,PaulMATHONCVinternship.docx,Paul MATHON BUSINESS STUDENT LOOKINF FOR AN IN...,[paulmathon0@gmail.com],Paul MATHON BUSINESS STUDENT LOOKINF FOR AN IN...,[],[+33 (0)6 86 18 04 57 ],[],[]


In [227]:
#!pip install commonregex

  Running setup.py bdist_wheel for commonregex ... done
  Stored in directory: /Users/jean-marcpicard/Library/Caches/pip/wheels/d1/33/d6/aca49143d5d350895d2a78a67980ddddfb1d047f2fed4cf2ce
Successfully built commonregex
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [229]:
address=[]
import commonregex
from commonregex import CommonRegex
#new_text=[]

for i in range(len(text)):
    #new_text.append(df['New_Text'][i+1])
    a=CommonRegex(df['New_Text'][i+1])
    #a=re.findall(r"((?i)\d+ ((?! \d+ ).)*{,40}(Singapore|France|Paris|Toulouse|Switzerland))", df['New_Text'][i+1])
    #a=re.findall(r"((?i)\d+ ((?! \d+ ).)*(rue|street|Avenue|Road|Boulevard|Drive)(, \d{5}| \d{5}|\b))", df['New_Text'][i+1])
    address.append(a.street_addresses)
print(address)

#parsed_text = CommonRegex()
#print(parsed_text.street_addresses)

[[], ['16 Spottiswoode Park Road,'], ['2016 External analyst'], ['2017    Team project', '2016   Innovation Project'], [], [], [], [], ['3 rue pierre de ronsard', '2008 1st'], [], [], ['2 nd   GRADE 2 nd  best'], []]


In [200]:
print(df['New_Text'][13])

L UCIE  PROST  Singapore LIFE SCIENCES ENGINEER Open to new opportunities in scientific and marketing project management E DUCATION 2012-2015  Engineering degree   from   ENSCM (Montpellier, France)   i.e. National Graduate School of Chemistry, a French   highly selective academic Institute of Science and Engineering.  Graduate Major:   Cosmetology and Pharmacology 2009-2011  Two years in Preparatory Classes at ENSCCF (Clermont-Ferrand, France) W ORK  E XPERIENCE 06/2017-03/2018 (09 months)  Sabbatical year traveling throughout South East Asia to discover and define in which   country we would settle down → Indonesia, Singapore, Malaysia, Thailand, Cambodia, Laos, Vietnam, Philippines, Sri Lanka 09/2015-05/2017 (18 months)  Business Manager , Pharmaceutical sector, CRO Aixial, Alten, Belgium Key Account Manager for GlaxoSmithKline (GSK) and for all clinical activities of Aixial, an Alten company Business: cold calling, client meetings, understanding and gathering client needs, finding 

In [102]:
#!pip install spacy

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [103]:
#!python -m spacy download en_core_web_sm

    100% |████████████████████████████████| 37.4MB 3.4MB/s ta 0:00:0111
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /Users/jean-marcpicard/anaconda3/lib/python3.6/site-packages/en_core_web_sm
    -->
    /Users/jean-marcpicard/anaconda3/lib/python3.6/site-packages/spacy/data/en_core_web_sm

    You can now load the model via spacy.load('en_core_web_sm')



In [104]:
#!python -m spacy download en_core_web_lg

    100% |████████████████████████████████| 852.3MB 9.1MB/s ta 0:00:0111  5% |█▋                              | 43.8MB 1.2MB/s eta 0:10:54    9% |███▏                            | 84.1MB 20.6MB/s eta 0:00:38    11% |███▊                            | 99.2MB 1.3MB/s eta 0:09:27    12% |███▉                            | 103.5MB 974kB/s eta 0:12:49    26% |████████▌                       | 225.8MB 4.3MB/s eta 0:02:26    27% |█████████                       | 237.7MB 6.7MB/s eta 0:01:3228% |█████████▏                      | 242.7MB 8.6MB/s eta 0:01:11    29% |█████████▌                      | 252.8MB 1.8MB/s eta 0:05:37    31% |██████████                      | 265.5MB 35.7MB/s eta 0:00:17    31% |██████████▏                     | 269.9MB 1.2MB/s eta 0:08:17    34% |███████████▏                    | 297.6MB 1.6MB/s eta 0:05:42    45% |██████████████▍                 | 384.2MB 5.4MB/s eta 0:01:28    48% |███████████████▍                | 410.9MB 17.3MB/s eta 0:00:26    50% |████████████████ 

    100% |████████████████████████████████| 39.8MB 9.2MB/s ta 0:00:0111 6% |██                              | 2.6MB 41.9MB/s eta 0:00:01
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /Users/jean-marcpicard/anaconda3/lib/python3.6/site-packages/fr_core_news_sm
    -->
    /Users/jean-marcpicard/anaconda3/lib/python3.6/site-packages/spacy/data/fr_core_news_sm

    You can now load the model via spacy.load('fr_core_news_sm')



In [106]:
#https://blog.ouseful.info/2017/09/04/simple-text-analysis-using-python-identifying-named-entities-tagging-fuzzy-string-matching-and-topic-modelling/
import spacy
nlp=spacy.load('en_core_web_sm')
example='Mathilde GABRIEL PROFILE Age : 26 Birthplace : Saint Cloud (92) Nationality : French Single Permis B YPY Member CONTACTS Mob. +65 9139 1049  16 Spottiswoode Park Road, Spottiswoode Suites, #26.01 088661 Singapore LANGUAGES French   +++++ English ++++ Spanish +++ SKILLS Creativity Dynamic Organized Team Hard- Quick Work Working Learner COMPETENCES Photoshop / Lightroom Wordpress iWorks : Pages / Keynote / Numbers Microsoft Office HOBBIES ABOUT ME Newly arrived to Singapore in October 2017 , I am a motivated, creative, and pas-sionate Communication & Marketing professional. I would like to offer my skills and energy to a large renowned and dynamic company.  WORK EXPERIENCE April 2017 - Now :  MARKETING MANAGER Lumion 3D  - Marina Baie des Anges - 06270 Villeneuve-Loubet Marketing Strategy  : Define all the major communication strategic lines - Marketing Plan & budgets for 2018 - Develop and Implement the planned marketing, communication and sales measures. Website  : Redesign the Wordpress website -  www.lumion3d.fr   - management & update Digital Marketing : Newsletters / Social Media / Content marketing (SEO) Product Launch  : Marketing campaigns Marketing Supports  : Design and creation of all the marketing documents, supports POS October 2016 - March 2017 :  TRAVEL BLOGGER During a 6 months trip around the word, I built and held a Wordpress : www.abbagliante.net Writing articles, page layout, publication and edition Video presentation of each destinations : Shooting, editing, postproduction and diffusion ( https://vimeo.com/user54571718 ) September 2013 - July 2016 :  MARKETING MANAGER Partnership (Thierry Voisin’s Holding) -  Quai Amiral Infernet, 06300 Nice The WOY Marketing Strategy  : Define all the major communication strategic lines for the brand’s visibility in a luxury industry, the yachting industry Total Rebranding  of the company in 2015 : New Corporate ID, new logo, new website… Charter & Sales Marketing  : Creation of all the marketing support for a fleet of 18 yachts. POS  : Creation of all the new point of sales advertising Event  : Organization of exhibitions and corporate events during Yachts Shows (MYBA Yachts show, MYS, Cannes Yachting Festival) and Open Days (ECPY) Newsletters  : Creation & Edition of quarterly newsletters YMCA Supporting the Rebranding  : New Corporate ID, new logo, new website.. work in collabora-tion with the operations manager and the graphic designer. Establishment of Communications Plans  : define targets, development of below-the-line communication, organization and following up events during yachts shows, advertising ma-nagement in yachting press. SOS Yachting Marketing Support  : Assist the French operations manager for all the marketing supports July 2015 - March 2016 :  COMMUNICATION CONSULTANT   Palace Shop  - 6 rue Martin Seytour, 06300 Nice Assisting the brand launch: define the communication & marketing strategy (Brandname, graphic charter, logotype,…), organisation of the inaugural evening Following up the social media - Community Management July 2010 - March 2015 :  GENERAL MANAGER LeBar’Oc  - 10 rue de Bonaparte, 06300 Nice Daily management Complete organization of weekly events and exceptional events with international DJs - Elaboration of Press Pack and following up of press impact - Managing of partnerships for summer musical events (Panda Event, Crossover) - Community Management.  EDUCATION 2014 - 2016 :  European Master in Communication & Marketing - FEDE - Nice - Specialty : Advertising Strategy and Digital Communication / external candidate 2013 - 2014 :  European Bachelor in Marketing - FEDE - Nice Graduated with honors 2011 - 2013  :   BTS - Small and Medium-sized Entreprise Management - Nice 2010 :  ILERI - International Relations & Geopolitics - Paris 17 e 2009 :  Baccalauréat ES - Saint Cloud (92) -   Mention Assez Bien'

doc=nlp(example)


In [1]:
def entities(doc):
    tags={}
    for entity in doc.ents:
        #print(entity.label, entity.label_, ' '.join(t.orth_ for t in entity))
        term=' '.join(t.orth_ for t in entity)
        if ' '.join(term) not in tags:
            tags[term]=[(entity.label, entity.label_)]
        else:
            tags[term].append((entity.label, entity.label_))
    print(tags)

In [108]:
a=entities(nlp(df['New_Text'][1]))
type(a)

{'Thanh Quan': [(378, 'PERSON')], 'French': [(379, 'NORP')], 'Vietnam': [(382, 'GPE')], 'River Valley Road': [(383, 'LOC')], '13': [(391, 'MONEY')], 'Valley Park Singapore': [(381, 'ORG')], '248368': [(388, 'DATE')], 'WhatsApp': [(381, 'ORG')], '65 8393 2616': [(388, 'DATE')], 'Management': [(381, 'ORG')], 'Caring': [(382, 'GPE')], 'Pastry': [(381, 'ORG')], 'Asian': [(379, 'NORP')], 'Perfection': [(381, 'ORG')], '2016': [(388, 'DATE')], 'Diploma': [(378, 'PERSON')], 'CAP': [(378, 'PERSON')], 'France': [(382, 'GPE')], 'High School of Culinary Art': [(381, 'ORG')], 'Hotel Expertise': [(381, 'ORG')], 'Paris': [(382, 'GPE')], '1994': [(388, 'DATE')], 'Master of Engineering in Information Technology France Institute': [(381, 'ORG')], 'the French Engineering School “ Ecole Centrale de Paris': [(381, 'ORG')], 'December 2016 - September 2017': [(388, 'DATE')], 'BOUILLON RACINE': [(385, 'EVENT')], '150': [(394, 'CARDINAL')], 'Laying': [(378, 'PERSON')], 'March 2016 -': [(388, 'DATE')], 'May 201

NoneType

In [109]:
#apply on the text without e-mail, shoud be also on the text without 
result=[]
person=[]
for i in range(len(text)):
    a=entities(nlp(df['New_Text'][i+1]))
    print(a)
    result.append(a)
len(result)

{'Thanh Quan': [(378, 'PERSON')], 'French': [(379, 'NORP')], 'Vietnam': [(382, 'GPE')], 'River Valley Road': [(383, 'LOC')], '13': [(391, 'MONEY')], 'Valley Park Singapore': [(381, 'ORG')], '248368': [(388, 'DATE')], 'WhatsApp': [(381, 'ORG')], '65 8393 2616': [(388, 'DATE')], 'Management': [(381, 'ORG')], 'Caring': [(382, 'GPE')], 'Pastry': [(381, 'ORG')], 'Asian': [(379, 'NORP')], 'Perfection': [(381, 'ORG')], '2016': [(388, 'DATE')], 'Diploma': [(378, 'PERSON')], 'CAP': [(378, 'PERSON')], 'France': [(382, 'GPE')], 'High School of Culinary Art': [(381, 'ORG')], 'Hotel Expertise': [(381, 'ORG')], 'Paris': [(382, 'GPE')], '1994': [(388, 'DATE')], 'Master of Engineering in Information Technology France Institute': [(381, 'ORG')], 'the French Engineering School “ Ecole Centrale de Paris': [(381, 'ORG')], 'December 2016 - September 2017': [(388, 'DATE')], 'BOUILLON RACINE': [(385, 'EVENT')], '150': [(394, 'CARDINAL')], 'Laying': [(378, 'PERSON')], 'March 2016 -': [(388, 'DATE')], 'May 201

{'Ponts de Cé': [(381, 'ORG')], '06': [(394, 'CARDINAL')], '74': [(394, 'CARDINAL')], '24 73 74': [(388, 'DATE')], 'B. Je': [(378, 'PERSON')], 'Pierrick JANVIER Curriculum': [(381, 'ORG')], 'Formations Septembre': [(378, 'PERSON')], 'd’entrée': [(381, 'ORG')], ' ': [(379, 'NORP'), (383, 'LOC'), (379, 'NORP'), (379, 'NORP'), (379, 'NORP'), (379, 'NORP'), (379, 'NORP'), (379, 'NORP'), (379, 'NORP'), (379, 'NORP'), (379, 'NORP'), (379, 'NORP')], 'École': [(379, 'NORP')], "d'Angers": [(379, 'NORP')], '5': [(394, 'CARDINAL')], '2014': [(388, 'DATE')], '2015': [(388, 'DATE')], 'Terminale ES  ': [(381, 'ORG')], 'Jean - Bodin': [(378, 'PERSON')], 'BAFA': [(381, 'ORG')], '  ': [(379, 'NORP')], 'Janvier 2017': [(388, 'DATE')], 'de la Sarl le Blue Monkeys': [(378, 'PERSON')], 'Décathlon': [(381, 'ORG')], '  TEST': [(381, 'ORG')], 'Formation   au métier de vendeurs': [(448, 'LAW')], 'Septembre': [(378, 'PERSON')], '2016': [(388, 'DATE')], 'Ponts de Cé : CDI': [(381, 'ORG')], 'CDD  ': [(379, 'NORP'

{'PROST': [(378, 'PERSON')], 'Singapore LIFE SCIENCES ENGINEER Open': [(381, 'ORG')], '2012 - 2015': [(388, 'DATE')], '  Engineering': [(381, 'ORG')], '   ENSCM': [(378, 'PERSON')], 'Montpellier': [(382, 'GPE')], 'France': [(382, 'GPE')], '   i.e. National Graduate School of Chemistry': [(381, 'ORG')], 'French': [(379, 'NORP')], '  ': [(379, 'NORP')], 'Institute of Science and Engineering': [(381, 'ORG')], ' ': [(381, 'ORG'), (379, 'NORP'), (383, 'LOC'), (379, 'NORP'), (379, 'NORP'), (379, 'NORP')], '2009 - 2011': [(388, 'DATE')], 'Two years': [(388, 'DATE')], 'ENSCCF': [(381, 'ORG')], 'Clermont - Ferrand': [(381, 'ORG')], 'E XPERIENCE': [(382, 'GPE')], '09 months': [(388, 'DATE')], 'South East Asia': [(383, 'LOC')], 'Indonesia': [(382, 'GPE')], 'Singapore': [(382, 'GPE')], 'Malaysia': [(382, 'GPE')], 'Thailand': [(382, 'GPE')], 'Cambodia': [(382, 'GPE')], 'Laos': [(382, 'GPE')], 'Vietnam': [(382, 'GPE')], 'Philippines': [(382, 'GPE')], 'Sri Lanka': [(382, 'GPE')], '05/2017': [(394, 'C

13

In [110]:
print(len(result))

13


In [111]:
name=[]


import nltk
!pip install nameparser
import nameparser
from nameparser.parser import HumanName


You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [112]:
#here not possible to have - in the tel
phone=[]
for i in range(len(text)):
    a=re.findall(r'[\+\(]?[0-9(\)][0-9 .\(\)]{8,}[0-9]', df['Text'][i+1])
    phone.append(a)
print(phone)
#needs to remove %
#needs to have a condition to remove date

[['+65 8393 2616', '5 5 6 7 8 8 7 6'], ['+65 9139 1049', '26.01 088661'], ['+33601392325'], ['+65 84387193', '+65 84387193'], ['06.31.21.19.67'], ['+33) 6 45 86 62 34'], ['+41 78 649 66 98'], ['06 74 24 73 74'], ['26.07.1990', '+33 6 69 65 81 79', '17.03.2014', '01.11.2017', '01.07.2013 06.09.2013', '01.10.2011 01.06.2013', '25.09.2013', '15.03.2014', '15.09.2011', '). 15.08.2014', '11.09.2009', '02.07.2011', '15.09.2008 1', '30.07.2009', '15.09.2007', '30.07.2008'], ['+33 (0)6 86 18 04 57'], ['+33 6 58 26 37 75'], ['+336 07 33 80 54'], ['+6586783019']]


In [125]:
from spacy.matcher import Matcher
nlp = spacy.load('en_core_web_sm')
matcher = Matcher(nlp.vocab)
patterns = [
        [{'ORTH': '('}, {'SHAPE': 'ddd'}, {'ORTH': ')'}, {'SHAPE': 'ddd'},
           {'ORTH': '-', 'OP': '?'}, {'SHAPE': 'ddd'}],
            [{'ORTH': '+'}, {'SHAPE': 'dd'}, {'SHAPE': 'dddd'}, {'SHAPE': 'dddd'}],
            [{'ORTH': '+'}, {'SHAPE': 'dddddddddd'}],
            [{'SHAPE': 'dd.dd'}]
           ]
matcher.add('PHONE', None, [{'ORTH': '('}, {'SHAPE': 'ddd'}, {'ORTH': ')'}, {'SHAPE': 'ddd'},
           {'ORTH': '-', 'OP': '?'}, {'SHAPE': 'dddddddddd'}],
            [{'ORTH': '+'}, {'SHAPE': 'ddddd'}],
            [{'ORTH': '+'}, {'SHAPE': 'dd'},{'SHAPE': 'dddd'},{'SHAPE': 'dddd'}],
            [{'ORTH': '('},{'ORTH': '+'}, {'SHAPE': 'dd'},{'ORTH': ')'},{'SHAPE': 'd dd dd dd dd'}],
            [{'ORTH': '+'}, {'SHAPE': 'dd'},{'SHAPE': 'd'},{'SHAPE': 'dd'},{'SHAPE': 'dd'},{'SHAPE': 'dd'},{'SHAPE': 'dd'}],
            [{'SHAPE': 'dd'},{'SHAPE': 'dd'},{'SHAPE': 'dd'},{'SHAPE': 'dd'},{'SHAPE': 'dd'}],
            [{'SHAPE': 'dd.dd.dd.dd.dd'}],
            [{'ORTH': '('},{'ORTH': '+'}, {'SHAPE': 'dd'},{'ORTH': ')'},{'SHAPE': 'dddd'},{'SHAPE': 'dddd'}],
            [{'ORTH': '('},{'ORTH': '+'}, {'SHAPE': 'dd'},{'ORTH': ')'},{'SHAPE': 'dddddddd'}],
            [{'ORTH': '+'}, {'SHAPE': 'dd.d.dd.dd.dd.dd'}]
           )

doc = nlp(u"Call me at (123) 456-789 or (123) 456 789! or +665")
phone_num=[]
for i in range(len(text)):
    matches = matcher(nlp(df['Text'][i+1]))
    for match_id, start, end in matches:
        if nlp(df['Text'][i+1])[start:end]!=None:
            span = nlp(df['Text'][i+1])[start:end]
            phone_num.append(span.text) 
        else:
            span='0'
            phone_num.append(span.text)    
phone_num

['+65 8393 2616',
 '+65 9139 1049',
 '06.31.21.19.67',
 '06 74 24 73 74',
 '+33 6 69 65 81 79',
 '+33 6 58 26 37 75']

In [142]:
nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])


def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords:
                lemmatized.append(lemma)
    return " ".join(lemmatized)


df['Text_After_Clean'] = df['Text'].apply(normalize, lowercase=True, remove_stopwords=True)
df

,Title,Text,Email,New_Text,Unit_postcode,Text_After_Clean
1,1805-TQL-CV-PastryChef.docx,LY Thanh Quan PASTRY CHEF Personal information...,[thanhquanly1001@gmail.com],LY Thanh Quan PASTRY CHEF Personal information...,"[#13-14, ]",
2,CV-MathildeGABRIEL.docx,Mathilde GABRIEL PROFILE Age : 26 Birthplace :...,[gabriel.mathilde@gmail.com],Mathilde GABRIEL PROFILE Age : 26 Birthplace :...,[#26.01 088661 ],
3,CV-RaphaëlSahakian-VE.docx,OBJECTIVE Management School student currently ...,[raphael.sahakian@gmail.com],OBJECTIVE Management School student currently ...,[],
4,CV_Louise.docx,334A Pasir Panjang Road #05-18 117382 Singap...,{louise.picard@student.ecp.fr},334A Pasir Panjang Road #05-18 117382 Singap...,{#05-18 117382 },
5,CVanglo.docx,Louis-Paul Courouble 1 rue des Flandres 59155 ...,[courouble.louispaul@orange.fr],Louis-Paul Courouble 1 rue des Flandres 59155 ...,[],
6,CVenanglais.docx,Hugo TORRENT 5 AVENUE DE LA MARCAISSONNE ...,[hugo.trrt31@gmail.com],Hugo TORRENT 5 AVENUE DE LA MARCAISSONNE ...,[],
7,CVF&B.docx,Giulia BALEGNO Current Address Les Roches Camp...,"{balegnogiulia@gmail.com, giulia.balegno@lesro...",Giulia BALEGNO Current Address Les Roches Camp...,[],
8,CVjanvierpierrick.docx,-81 rue David D’Angers 49130 les Ponts de Cé. ...,[Pierrick.janvier@essca.eu],-81 rue David D’Angers 49130 les Ponts de Cé. ...,[],
9,oussama-CVENG.docx,KHOMSI Oussama SOFTWARE CONCEPTION & DEVELOPME...,[okhomsielhaddad@gmail.com],KHOMSI Oussama SOFTWARE CONCEPTION & DEVELOPME...,[],
10,PaulMATHONCVinternship.docx,Paul MATHON BUSINESS STUDENT LOOKINF FOR AN IN...,[paulmathon0@gmail.com],Paul MATHON BUSINESS STUDENT LOOKINF FOR AN IN...,[],


In [116]:
from nltk import sent_tokenize
sentences=sent_tokenize(df['New_Text'][1])

from nltk.tokenize import word_tokenize
tokens = word_tokenize(df['New_Text'][1])
tokens = [w.lower() for w in tokens]
print(tokens[:100])

words = [word for word in tokens if word.isalpha()]
print(words[:100])


['ly', 'thanh', 'quan', 'pastry', 'chef', 'personal', 'information', 'french', 'citizenship', ',', 'born', 'in', 'vietnam', '483b', 'river', 'valley', 'road', 'valley', 'park', 'singapore', '248368', 'whatsapp', ':', 'key', 'skills', 'management', 'and', 'thinking', 'ahead', 'enjoying', 'being', 'dynamic', ',', 'organized', ',', 'creative', 'and', 'passionate', 'caring', 'about', 'health', '&', 'safety', 'by', 'implementing', 'haccp', 'being', 'autonomous', 'but', 'enjoying', 'working', 'within', 'a', 'team', 'additional', 'information', 'gastronomy', 'photography', 'profil', 'summary', 'a', 'former', 'it', 'project', 'manager', 'who', 'undertook', 'a', 'retraining', 'educational', 'program', 'in', 'pastry', ',', 'a', 'discipline', 'which', 'i', 'have', 'always', 'been', 'very', 'keen', 'on', 'this', 'educational', 'program', 'has', 'allowed', 'me', 'to', 'develop', 'my', 'techniques', ',', 'master', 'basics', 'and', 'develop', 'skills']
['ly', 'thanh', 'quan', 'pastry', 'chef', 'perso

In [ ]:
import nltk

import pprint
def ie_preprocess(document):
    sentences = nltk.sent_tokenize(document) 
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]